In [2]:
import xarray as xr
import sys
import matplotlib.pyplot as plt
import os
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
#from tensorflow.keras.callbacks import Callback
import numpy as np
#import tensorflow as tf
from functools import partial

#AUTOTUNE = tf.data.experimental.AUTOTUNE
#from dask.diagnostics import ProgressBar
import pandas as pd
#import tensorflow.keras.layers as layers
import json
#from tensorflow.keras.optimizers import Adam
#from tensorflow.distribute import MirroredStrategy
#from tensorflow.keras import layers

# custom modules
sys.path.append(os.getcwd())
from src.layers import *
from src.models import *
from src.gan import *
from src.process_input_training_data import *




2025-05-15 01:47:32.877385: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-15 01:47:47.081307: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
config_file = '/nesi/project/niwa00018/queenle/ML_emulator_development/emulators/pr_ACCESS-CM2_2080-2099/config_info.json'
pretrained_unet = False  # set this as true if you want to use the same U-Net or a specific unet everytime.
with open(config_file, 'r') as f:
    config = json.load(f)

# set to 1 if using normal precip (kgs-1)
input_shape = config["input_shape"]  # the input shape of the reanalyses
output_shape = config["output_shape"]
# modified the output channels, filters, and output shape
n_filters = config["n_filters"]
kernel_size = config["kernel_size"]
n_channels = config["n_input_channels"]
n_output_channels = config["n_output_channels"]
BATCH_SIZE = 16#config["batch_size"]
init_weights = True
# config["itensity_weight"] = config["itensity_weight"]*2

try:
    temp_sampling_type = config['temp_sampling_type']
    samplings_file = config['sampling_file']
    samplings_df = pd.read_csv(samplings_file)
except:
    temp_sampling_type = 'all'
    sampling_file = None




In [4]:
# creating a path to store the model outputs
if not os.path.exists(f'{config["output_folder"]}/{config["model_name"]}'):
    os.makedirs(f'{config["output_folder"]}/{config["model_name"]}')
    

gcms_for_training = config["gcms_for_training_GAN"]
stacked_X, y, vegt, orog, he = preprocess_input_data(config)
output_means = xr.open_dataset(config["means_output"])
output_stds = xr.open_dataset(config["stds_output"])
# To modify
#stacked_X = stacked_X.sel(GCM =config["gcms_for_training_GAN"][0])
#y = y.sel(GCM =config["gcms_for_training_GAN"][0])[['pr']]

stacked_X = stacked_X.sel(GCM=gcms_for_training)
y = y.sel(GCM=gcms_for_training)

try:
    output_varname = [config["output_varname"]]
except:
    output_varname = ["pr"]
    
if output_varname[0] =="pr":
    config["delta"] =1
    conversion_factor = 3600 * 24
    config['conversion_factor'] = conversion_factor 
    y[output_varname[0]] = np.log(y[output_varname[0]] * conversion_factor + 1)
    loss_weight =1
elif "sfcWind" in output_varname[0]:   
    y[output_varname[0]] = (y[output_varname[0]] - output_means['sfcWind'].mean()) / output_stds['sfcWind'].mean()
    loss_weight = 10
else:
    y[output_varname[0]] = (y[output_varname[0]] - output_means[output_varname[0]].mean()) / output_stds[output_varname[0]].mean()
    loss_weight = 10

common_times = stacked_X.time.to_index().intersection(y.time.to_index())
y = y.sel(time=common_times)



In [7]:
# select temporal samplings (temp_sampling = ['all',first_10','last_30','random_20']
if temp_sampling_type != 'all':
    
    sample_type = temp_sampling_type.split('_')[0] 
    
    if sample_type == 'range':
        period = temp_sampling_type.split('_')[1]
        start,end = period.split('-')
        stacked_X = stacked_X.sel(time=slice(start,end))
        y = y.sel(time=slice(start,end))
        
    elif sample_type == 'random':
        sample_n = temp_sampling_type.split('_')[1]
        years = [int(val) for val in samplings_df[sample_n] if not np.isnan(val)]
        stacked_X = stacked_X.where(stacked_X.time.dt.year.isin(years),drop=True)
        y = y.where(y.time.dt.year.isin(years),drop=True)


y = y[output_varname[0]].transpose("time", "lat", "lon", "GCM")
stacked_X = stacked_X.transpose("time", "lat", "lon", "GCM", "channel")

In [8]:
y

<xarray.DataArray 'pr' (time: 7300, lat: 172, lon: 179, GCM: 1)>
array([[[[3.26856379e-01],
         [3.16913549e-01],
         [3.08097846e-01],
         ...,
         [4.98921175e-02],
         [1.09964294e-01],
         [2.10029313e-01]],

        [[3.29459238e-01],
         [3.21114497e-01],
         [3.14668698e-01],
         ...,
         [2.94414854e-02],
         [2.91314259e-02],
         [2.96031246e-02]],

        [[3.55861196e-01],
         [3.50347970e-01],
         [3.44708547e-01],
         ...,
...
         ...,
         [1.04357366e-05],
         [1.67209287e-06],
         [6.18394302e-06]],

        [[6.38126951e-01],
         [7.51005737e-01],
         [8.21357882e-01],
         ...,
         [1.02110843e-04],
         [8.73618558e-06],
         [8.06281828e-06]],

        [[6.72781852e-01],
         [7.69133071e-01],
         [8.28056589e-01],
         ...,
         [5.23269119e-04],
         [1.09895467e-04],
         [2.50132384e-05]]]])
Coordinates:
  * time     (time) datetime64[ns] 2080-01-01 2080-01-02 ... 2099-12-31
  * lon      (lon) float32 164.9 165.0 165.1 165.2 ... 183.6 183.7 183.9 184.0
  * lat      (lat) float32 -51.21 -51.11 -51.0 -50.89 ... -33.07 -32.97 -32.86
  * GCM      (GCM) object 'ACCESS-CM2'

In [9]:
stacked_X

<xarray.DataArray 'q_500' (time: 7300, lat: 23, lon: 26, GCM: 1, channel: 8)>
array([[[[[-1.04166484e+00, -8.09694409e-01,  1.22149158e+00, ...,
            5.63617826e-01, -1.29353225e+00, -1.85940981e+00]],

         [[-1.04011202e+00, -7.72200644e-01,  1.29696810e+00, ...,
            6.06199026e-01, -1.27764702e+00, -1.76352501e+00]],

         [[-1.03736913e+00, -7.85724282e-01,  1.36869907e+00, ...,
            6.38164461e-01, -1.27881467e+00, -1.72206748e+00]],

         ...,

         [[-3.27602059e-01, -1.16059744e+00,  4.16561931e-01, ...,
           -1.94297969e-01, -1.55241287e+00, -2.14623880e+00]],

         [[-3.09233785e-01, -1.21282029e+00,  4.19859052e-01, ...,
           -7.33152926e-02, -1.63470972e+00, -2.04973173e+00]],

         [[-3.12274933e-01, -1.26866746e+00,  4.63288933e-01, ...,
           -4.12866399e-02, -1.68058753e+00, -1.96103919e+00]]],

...
        [[[ 1.68889239e-02,  5.53673506e+00,  3.39368701e-01, ...,
           -3.73002619e-01,  2.73149133e+00,  3.24010324e+00]],

         [[ 1.57354522e+00,  4.22788382e+00,  3.10208976e-01, ...,
           -4.95640785e-01,  2.72775602e+00,  3.71116209e+00]],

         [[ 3.35647702e+00,  4.19392395e+00, -4.78332601e-02, ...,
           -8.93146753e-01,  2.63751221e+00,  3.70481491e+00]],

         ...,

         [[-7.51735508e-01,  8.11155736e-02, -2.56430745e+00, ...,
           -4.77445349e-02,  2.33319068e+00,  2.76786518e+00]],

         [[-6.30344808e-01,  4.84397322e-01, -2.60826612e+00, ...,
           -3.25802155e-02,  2.37390494e+00,  2.67932343e+00]],

         [[-5.86808503e-01,  5.66306770e-01, -2.49688220e+00, ...,
           -4.06393819e-02,  2.42418480e+00,  2.64462590e+00]]]]],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2080-01-01 2080-01-02 ... 2099-12-31
  * lon      (lon) float64 150.6 152.1 153.6 155.1 ... 183.6 185.1 186.6 188.1
  * lat      (lat) float64 -59.38 -57.88 -56.38 -54.88 ... -29.38 -27.88 -26.38
  * GCM      (GCM) object 'ACCESS-CM2'
  * channel  (channel) <U5 'q_500' 'q_850' 'u_500' ... 'v_850' 't_500' 't_850'